# TSTR GM Dataset B - Cardio

In [1]:
#import libraries
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import os
print('Libraries imported!!')

Libraries imported!!


In [2]:
#define directory of functions and actual directory
HOME_PATH = '' #home path of the project
FUNCTIONS_DIR = 'EVALUATION FUNCTIONS/UTILITY'
ACTUAL_DIR = os.getcwd()

#change directory to functions directory
os.chdir(HOME_PATH + FUNCTIONS_DIR)

#import functions for data labelling analisys
from utility_evaluation import DataPreProcessor
from utility_evaluation import train_evaluate_model

#change directory to actual directory
os.chdir(ACTUAL_DIR)
print('Functions imported!!')

Functions imported!!


## 1. Read data

In [3]:
#read real dataset
train_data = pd.read_csv(HOME_PATH + 'SYNTHETIC DATASETS/GM/B_Cardio_Data_Synthetic_GM.csv')
categorical_columns = ['gender','cholesterol','gluc','smoke','alco','active','cardio']
for col in categorical_columns :
    train_data[col] = train_data[col].astype('category')
train_data

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,3540,14298,2,159,68.646934,141,97,1,1,0,0,1,0
1,8670,22551,1,149,72.531185,131,74,3,1,0,0,1,1
2,26537,22729,1,148,74.919117,140,108,2,1,0,0,1,1
3,16126,19467,1,160,86.949278,123,90,1,1,0,0,0,1
4,39966,22059,1,171,83.258253,116,81,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
55995,61086,20098,1,156,60.735778,128,92,1,1,0,1,0,1
55996,95276,20522,2,154,88.455547,125,109,2,1,0,0,1,1
55997,49638,18805,1,165,79.951021,171,138,2,1,0,0,1,1
55998,30191,18800,2,164,66.637624,109,71,2,2,1,1,1,0


In [4]:
#read test data
test_data = pd.read_csv(HOME_PATH + 'REAL DATASETS/TEST DATASETS/B_Cardio_Data_Real_Test.csv')
for col in categorical_columns :
    test_data[col] = test_data[col].astype('category')
test_data

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,66728,21770,1,156,64.0,140,80,2,1,0,0,1,1
1,69098,21876,1,170,85.0,160,90,1,1,0,0,1,1
2,59185,23270,1,151,90.0,130,80,1,1,0,0,1,1
3,49288,19741,1,159,97.0,120,80,1,1,0,0,1,1
4,62481,18395,1,164,68.0,120,80,1,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13995,30751,20490,2,172,70.0,120,80,1,1,0,0,0,1
13996,23253,16797,2,174,96.0,120,80,1,2,0,0,1,1
13997,34847,22607,1,165,66.0,110,80,1,1,0,0,0,0
13998,40118,19670,1,157,89.0,120,80,3,3,0,0,1,1


In [5]:
target = 'cardio'
#quick look at the breakdown of class values
print('Train data')
print(train_data.shape)
print(train_data.groupby(target).size())
print('#####################################')
print('Test data')
print(test_data.shape)
print(test_data.groupby(target).size())

Train data
(56000, 13)
cardio
0    28085
1    27915
dtype: int64
#####################################
Test data
(14000, 13)
cardio
0    6988
1    7012
dtype: int64


## 2. Pre-process training data

In [6]:
target = 'cardio'
categorical_columns = ['gender','cholesterol','gluc','smoke','alco','active']
numerical_columns = numerical_columns = train_data.select_dtypes(include=['int64']).columns.tolist()
categories = [np.array(range(2)), np.array(range(3)), np.array(range(3)), np.array(range(2)), np.array(range(2)), 
            np.array(range(2))]


data_preprocessor = DataPreProcessor(categorical_columns, numerical_columns, categories)
x_train = data_preprocessor.preprocess_train_data(train_data.loc[:, train_data.columns != target])
y_train = train_data.loc[:, target]

x_train.shape, y_train.shape

((56000, 19), (56000,))

## 3. Preprocess test data

In [7]:
x_test = data_preprocessor.preprocess_test_data(test_data.loc[:, test_data.columns != target])
y_test = test_data.loc[:, target]
x_test.shape, y_test.shape

((14000, 19), (14000,))

## 4. Create a dataset to save the results

In [8]:
results = pd.DataFrame(columns = ['model','accuracy','precision','recall','f1'])
results

,model,accuracy,precision,recall,f1


## 4. Train and evaluate Random Forest Classifier

In [9]:
rf_results = train_evaluate_model('RF', x_train, y_train, x_test, y_test)
results = results.append(rf_results, ignore_index=True)
rf_results

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    4.6s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    9.8s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.2s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.4s finished


,model,accuracy,precision,recall,f1
0,RF,0.6913,0.6916,0.6913,0.6911


## 5. Train and Evaluate KNeighbors Classifier

In [10]:
knn_results = train_evaluate_model('KNN', x_train, y_train, x_test, y_test)
results = results.append(knn_results, ignore_index=True)
knn_results

,model,accuracy,precision,recall,f1
0,KNN,0.6639,0.6662,0.6639,0.6628


## 6. Train and evaluate Decision Tree Classifier

In [11]:
dt_results = train_evaluate_model('DT', x_train, y_train, x_test, y_test)
results = results.append(dt_results, ignore_index=True)
dt_results

,model,accuracy,precision,recall,f1
0,DT,0.5915,0.5919,0.5915,0.591


## 7. Train and evaluate Support Vector Machines Classifier

In [12]:
svm_results = train_evaluate_model('SVM', x_train, y_train, x_test, y_test)
results = results.append(svm_results, ignore_index=True)
svm_results

[LibSVM]

,model,accuracy,precision,recall,f1
0,SVM,0.5014,0.5012,0.5014,0.4893


## 8. Train and evaluate Multilayer Perceptron Classifier

In [13]:
mlp_results = train_evaluate_model('MLP', x_train, y_train, x_test, y_test)
results = results.append(mlp_results, ignore_index=True)
mlp_results

Iteration 1, loss = 0.61819761
Iteration 2, loss = 0.61329068
Iteration 3, loss = 0.61254985
Iteration 4, loss = 0.61213454
Iteration 5, loss = 0.61180809
Iteration 6, loss = 0.61118171
Iteration 7, loss = 0.61070403
Iteration 8, loss = 0.61039662
Iteration 9, loss = 0.61026354
Iteration 10, loss = 0.61002671
Iteration 11, loss = 0.60955571
Iteration 12, loss = 0.60929629
Iteration 13, loss = 0.60873465
Iteration 14, loss = 0.60860383
Iteration 15, loss = 0.60771089
Iteration 16, loss = 0.60736411
Iteration 17, loss = 0.60732555
Iteration 18, loss = 0.60673639
Iteration 19, loss = 0.60607919
Iteration 20, loss = 0.60504011
Iteration 21, loss = 0.60518320
Iteration 22, loss = 0.60412051
Iteration 23, loss = 0.60363053
Iteration 24, loss = 0.60299843
Iteration 25, loss = 0.60227453
Iteration 26, loss = 0.60164542
Iteration 27, loss = 0.60082596
Iteration 28, loss = 0.60056883
Iteration 29, loss = 0.59958381
Iteration 30, loss = 0.59917517
Iteration 31, loss = 0.59814336
Iteration 32, los

,model,accuracy,precision,recall,f1
0,MLP,0.6646,0.6658,0.6646,0.664


## 9. Save results file

In [14]:
results.to_csv('RESULTS/models_results_gm.csv', index=False)
results

,model,accuracy,precision,recall,f1
0,RF,0.6913,0.6916,0.6913,0.6911
1,KNN,0.6639,0.6662,0.6639,0.6628
2,DT,0.5915,0.5919,0.5915,0.5910
3,SVM,0.5014,0.5012,0.5014,0.4893
4,MLP,0.6646,0.6658,0.6646,0.6640
